# 신경망 모델 구성하기

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### 클래스 정의하기

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


### 모델 계층(Layer)

In [9]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [10]:
# nn.Flatten

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [11]:
# nn.Linear

layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
# nn.ReLU

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0306, -0.6617, -0.1332, -0.0870, -0.1126, -0.1323,  0.9918, -0.3469,
          0.1682,  0.1799,  0.6942, -0.1228,  0.0178, -0.1200,  0.0274, -0.0306,
         -0.5949,  0.1423, -0.0107,  0.1472],
        [ 0.0260, -1.0427, -0.2169, -0.0191,  0.0554,  0.0659,  0.5268, -0.2862,
          0.2389,  0.0404,  0.5175, -0.3873, -0.1350, -0.1758,  0.2911, -0.5512,
         -0.6966,  0.1751, -0.0535,  0.5622],
        [ 0.0383, -0.6137,  0.1644, -0.1995, -0.0587, -0.1530,  0.6952, -0.4682,
         -0.0936, -0.0521,  0.4061,  0.1538,  0.1211, -0.3642, -0.2853, -0.4476,
         -0.5112,  0.3202, -0.0383,  0.2038]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9918, 0.0000, 0.1682,
         0.1799, 0.6942, 0.0000, 0.0178, 0.0000, 0.0274, 0.0000, 0.0000, 0.1423,
         0.0000, 0.1472],
        [0.0260, 0.0000, 0.0000, 0.0000, 0.0554, 0.0659, 0.5268, 0.0000, 0.2389,
         0.0404, 0.5175, 0.0000, 0.0000, 0.0000, 0.29

In [13]:
# nn.Sequential

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [16]:
# nn.Softmax

softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수

In [20]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()}\n")
    #print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784])

Layer: linear_relu_stack.0.bias | Size: torch.Size([512])

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512])

Layer: linear_relu_stack.2.bias | Size: torch.Size([512])

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512])

Layer: linear_relu_stack.4.bias | Size: torch.Size([10])

